In [1]:
import random
import time
import math
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import preprocess
from DataLoader import DataLoader
import model

In [2]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

True


## Data loading and processing

In [3]:
# twitter dataset

PAD_token = 0
SOS_token = 1
EOS_token = 2
UNK_token = 3

max_vocab_size = 20000
max_sen, min_sen = 14, 3
unk_most = 2
reverse_flag = 1  # reverse the input sequence order Sutskever et al., 2014
inverse_flag = 0  # MMI bidirection: train P(T|S) by inversing source and target

dataStat = preprocess.dataPreProcess('dataset/twitter.txt', max_vocab_size, max_sen, min_sen, unk_most, reverse_flag, inverse_flag) 
print("Number of total words:", dataStat.numOfWords)

wordCount = sorted(dataStat.word2cnt.values(), reverse=True)
print("Dictionary cover ratio:", sum(wordCount[:max_vocab_size-4]) / sum(wordCount))

Number of total words: 51805
Dictionary cover ratio: 0.9796814762021151


In [4]:
# pad or cut input and output sentence

def padding(source, maxLen):
    return np.pad(source[:maxLen],(0,max(0,maxLen-len(source))),'constant')

input_max_len, output_max_len = 15, 15

pairsNum = len(dataStat.pairsInd)
pairsLength = np.array([[len(l[0]), len(l[1])] for l in dataStat.pairsInd])
upperLength = np.concatenate((np.ones((pairsNum,1), dtype=int)*input_max_len, 
                              np.ones((pairsNum,1), dtype=int)*output_max_len), axis=1)
pairsLength = np.minimum(pairsLength,upperLength)
pairsAligned = np.array([np.concatenate((padding(l[0], input_max_len), 
                                              padding(l[1], output_max_len))) for l in dataStat.pairsInd])

In [5]:
train_type = 'resume'

ratios = [0.90, 0.05, 0.05]
pairsNumTrain, pairsNumDeve = int(ratios[0]*pairsNum), int(ratios[1]*pairsNum)
pairsNumTest = pairsNum - pairsNumTrain - pairsNumDeve

if train_type=='restart':
    deve_idxes = np.random.choice(pairsNum, pairsNumDeve, replace=False)
    test_idxes = np.random.choice(list(set(np.arange(pairsNum)).difference(set(deve_idxes))), pairsNumTest, replace=False)
    train_idxes = np.array(list(set(np.arange(pairsNum)).difference(set(deve_idxes)).difference(set(test_idxes))))
    np.save("parameter/pairsIdxesTriple.npy", (train_idxes, deve_idxes, test_idxes))
else:
    train_idxes, deve_idxes, test_idxes = np.load( "parameter/pairsIdxesTriple.npy" )

pairsAlignedTrain, pairsAlignedDeve, pairsAlignedTest = pairsAligned[train_idxes], pairsAligned[deve_idxes], pairsAligned[test_idxes]
pairsLengthTrain, pairsLengthDeve, pairsLengthTest = pairsLength[train_idxes], pairsLength[deve_idxes], pairsLength[test_idxes]
    
print("Total training pairs: ",pairsNumTrain)
print("Total develop pairs: ",pairsNumDeve)
print("Total test pairs: ",pairsNumTest)

Total training pairs:  104010
Total develop pairs:  5778
Total test pairs:  5779


## Construct Seq2Seq model

In [6]:
# A tiny verification program

# environment setup
vocab_size = min(dataStat.numOfWords, max_vocab_size)
hidden_size = 1000
train_type = 'resume'

embedding = nn.Embedding(vocab_size, hidden_size)
rnnEncoder = model.Encoder(embedding, vocab_size,20,hidden_size, n_layers=2, bidirectional=False, variable_lengths=True)
rnnDecoder = model.Decoder(embedding, vocab_size,1,hidden_size, n_layers=2)

if train_type.lower()=='restart': pass
elif train_type.lower()=='resume':
    para_name = 'twitter_0518_35'
    embedding.load_state_dict(torch.load('parameter/embeding_'+para_name+'.pt'))
    rnnEncoder.load_state_dict(torch.load('parameter/encoder_'+para_name+'.pt'))
    rnnDecoder.load_state_dict(torch.load('parameter/decoder_'+para_name+'.pt'))
else: print("Please enter valid training type !")

if USE_CUDA:
    rnnEncoder.cuda()
    rnnDecoder.cuda()

criterion = nn.NLLLoss(size_average=True)

In [7]:
learning_rate = 0.0002
optimizer_encoder = optim.Adam(rnnEncoder.parameters(), learning_rate)
optimizer_decoder = optim.Adam(rnnDecoder.parameters(), learning_rate)

# initialize dataloader
batch_size = 32
trainLoader = DataLoader(pairsAlignedTrain, pairsLengthTrain, input_max_len, output_max_len)
trainLoader.reset(batch_size)

deveLoader = DataLoader(pairsAlignedDeve, pairsLengthDeve, input_max_len, output_max_len)
deveLoader.reset(1)

testLoader = DataLoader(pairsAlignedTest, pairsLengthTest, input_max_len, output_max_len)
testLoader.reset(1)

print("iteration per epoch:", int(pairsNumTrain/batch_size))


iteration per epoch: 3250


In [8]:
def geneMask(outputs_record, lengths):
    batch_size = lengths.size(0)
    # prepare
    comp = torch.arange(output_max_len).view(-1,1)
    if USE_CUDA: comp = comp.cuda()
    comp_ex = comp.repeat(1,vocab_size).repeat(batch_size,1,1)
    # generate
    l_ex = lengths[:,1].view(batch_size,-1).repeat(1,vocab_size).view(batch_size,1,-1)
    if USE_CUDA: l_ex = l_ex.type(torch.cuda.FloatTensor)
    else: l_ex = l_ex.type(torch.FloatTensor)
    mask = comp_ex < l_ex
    if USE_CUDA: mask = mask.type(torch.cuda.FloatTensor)
    else: mask = mask.type(torch.FloatTensor)
    return torch.mul(mask, outputs_record)

## Train

In [9]:
def oneEpoch():

    running_loss = 0

    for batch_ind in range(int(pairsNum/batch_size)+1):
    
        # prepare mini-batch data
        try:
            inputs, targets, lengths = trainLoader.getMiniBatch()
        except Exception as e:
            # print('GG...')
            break
        else:
            # print('Good!')
            # Zero gradients of both optimizers
            optimizer_encoder.zero_grad()
            optimizer_decoder.zero_grad()

            # encoding and decoding
            inputs, targets = Variable(inputs), Variable(targets)
            hid_init = rnnEncoder.init_hidden(batch_size)
            out_enc, hid_enc = rnnEncoder.forward(inputs,lengths[:,0],hid_init)
            
            # teacher forcing
            hid_dec = hid_enc#[:rnnDecoder.n_layers]

            # SOS_token
            in_dec = Variable(torch.LongTensor([dataStat.word2ind['SOS']] * batch_size))
            if USE_CUDA: in_dec = in_dec.cuda()
            outputs_record, hid_dec = rnnDecoder.forward(in_dec.view(batch_size,-1),hid_dec)
            # continue
            for i in range(output_max_len-1):
                out_dec, hid_dec = rnnDecoder.forward(targets[:,i].view(batch_size,-1),hid_dec)
                outputs_record = torch.cat((outputs_record, out_dec), 1)

            outputs_mask = geneMask(outputs_record, lengths)
            loss = criterion(torch.transpose(outputs_mask,1,2), targets)
            # print(loss)
            
            loss.backward()
            optimizer_encoder.step()
            optimizer_decoder.step()
            
            running_loss += float(loss)
            
            if (batch_ind+1)%1000 == 0:
                print("iteration", batch_ind+1, " ---- running loss:", running_loss/batch_ind)
            
    # print('running_loss:',running_loss)
    return running_loss/batch_ind

In [10]:
def savePara(epoch):
    para_name = 'twitter_0518_'+str(epoch)
    torch.save(embedding.state_dict(),'parameter/embeding_'+para_name+'.pt')
    torch.save(rnnEncoder.state_dict(),'parameter/encoder_'+para_name+'.pt')
    torch.save(rnnDecoder.state_dict(),'parameter/decoder_'+para_name+'.pt')

In [21]:
print("Begin training...")
print(time.asctime( time.localtime(time.time()) ))

for i in range(10):
    trainLoader.reset(batch_size)
    loss = oneEpoch()
    if (i+1)%1==0:
        print('Epoch:', i+1, '\tLoss:',loss)
        print(time.asctime( time.localtime(time.time()) ))
    if (i+1)%5==0:
        savePara(i+1+30)

Begin training...
Fri May 18 13:48:28 2018
iteration 1000  ---- running loss: 0.7764374853015781
iteration 2000  ---- running loss: 0.7936298665313377
iteration 3000  ---- running loss: 0.8099324350636894
Epoch: 1 	Loss: 0.8131477450590867
Fri May 18 13:53:55 2018
iteration 1000  ---- running loss: 0.7073472350447982
iteration 2000  ---- running loss: 0.7228136096315542
iteration 3000  ---- running loss: 0.7357599485632021
Epoch: 2 	Loss: 0.7386495542526245
Fri May 18 13:59:20 2018
iteration 1000  ---- running loss: 0.6406484229547961
iteration 2000  ---- running loss: 0.6553890522030844
iteration 3000  ---- running loss: 0.6696273117869963
Epoch: 3 	Loss: 0.6734025310736436
Fri May 18 14:04:46 2018
iteration 1000  ---- running loss: 0.5867972803545428
iteration 2000  ---- running loss: 0.597785772011243
iteration 3000  ---- running loss: 0.6119136117962688
Epoch: 4 	Loss: 0.615221876676266
Fri May 18 14:10:12 2018
iteration 1000  ---- running loss: 0.5322195835657664
iteration 2000  -

## Build Language Model: P(T)

In [29]:
# construct P(target) language model

freqLM = {}

for i in range(pairsNumTrain):
    length = pairsLengthTrain[i][1]
    rsps = pairsAlignedTrain[i][input_max_len:input_max_len+length]
    dic = freqLM
    for j in range(min(5,length)):
        if rsps[j] not in dic: dic[rsps[j]] = [1,{}]
        else: dic[rsps[j]][0] += 1
        dic = dic[rsps[j]][1]


In [30]:
def conProb(prefix):
    dic = freqLM
    dist_array = np.ones(vocab_size)
    try:
        for ind in prefix:
            dic = dic[ind][1]
    except Exception as e:
        pass
    else:
        for key in dic:
            dist_array[key] += dic[key][0]
    total_freq = np.sum(dist_array)
    dist_tensor = torch.FloatTensor(np.log(dist_array/total_freq))
    if USE_CUDA: dist_tensor = dist_tensor.cuda()
    return dist_tensor

## Evalute by BLEU and distinct

In [31]:
# translate into natural language

def showResult(ind_seq, reverse=False):
    token_list = []
    for i in ind_seq:
        if i == dataStat.word2ind['EOS']: break
        token_list.append(dataStat.ind2word[i])
    return ' '.join(token_list[::-1]) if reverse else  ' '.join(token_list)

In [32]:
def topOneDecode(decoder, decoder_hidden, stat, max_length=output_max_len):
    
    decoder_input = torch.LongTensor([SOS_token]).view(1,-1)
    if USE_CUDA: 
        decoder_input = decoder_input.cuda()
        decoder_hidden = decoder_hidden.cuda()

    decoded_words = []

    for di in range(max_length):

        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append(ni.item())
            break
        else:
            decoded_words.append(ni.item())

        decoder_input = torch.LongTensor([[ni]])
        if USE_CUDA: decoder_input = decoder_input.cuda()

    return decoded_words

In [33]:
def beamDecode(decoder, decoder_hidden, voc, beam_size, lamda, threshold, max_length=output_max_len):
    terminal_sentences, prev_top_sentences, next_top_sentences = [], [], []
    prev_top_sentences.append(Sentence(decoder_hidden))
    for _ in range(max_length):
        for sentence in prev_top_sentences:
            decoder_input = torch.LongTensor([[sentence.last_idx]])
            if USE_CUDA: decoder_input = decoder_input.cuda()
            decoder_output, decoder_hidden = decoder(decoder_input, sentence.decoder_hidden)
            
            # apply MMI anti-language model
            if len(sentence.sentence_idxes)<threshold:
                LM_output = conProb([int(idx) for idx in sentence.sentence_idxes])
                decoder_output -= lamda*LM_output.view(1,1,-1)
            
            topv, topi = decoder_output.topk(beam_size)
            term, top = sentence.addTopk(topi, topv, decoder_hidden, beam_size, voc)
            terminal_sentences.extend(term)
            next_top_sentences.extend(top)
            
        next_top_sentences.sort(key=lambda s: s.getScore(), reverse=True)
        prev_top_sentences = next_top_sentences[:beam_size]
        next_top_sentences = []

    terminal_sentences += [sentence.toWordScore(voc) for sentence in prev_top_sentences]
    terminal_sentences.sort(key=lambda x: x[1], reverse=True)

    n = min(len(terminal_sentences), 10)  # N-best list
    return terminal_sentences[:n]

In [34]:
class Sentence:
    def __init__(self, decoder_hidden, last_idx=SOS_token, sentence_idxes=[], sentence_scores=[]):
        if(len(sentence_idxes) != len(sentence_scores)):
            raise ValueError("length of indexes and scores should be the same")
        self.decoder_hidden = decoder_hidden
        self.last_idx = last_idx
        self.sentence_idxes =  sentence_idxes
        self.sentence_scores = sentence_scores

    def getScore(self, mode='sum', gamma=0.2):
        if len(self.sentence_scores) == 0:
            print("sentence of length 0")
            return torch.cuda.FloatTensor(-999) if USE_CUDA else torch.FloatTensor(-999)
        if mode=='avg':
            res = sum(self.sentence_scores) / len(self.sentence_scores)
        else:
            res = sum(self.sentence_scores) + gamma*len(self.sentence_scores)
        return res


    def addTopk(self, topi, topv, decoder_hidden, beam_size, voc):
        terminates, sentences = [], []
        
        topi, topv = topi[0], topv[0]  # get data out of batch
        
        for i in range(beam_size):
            if topi[0][i] == EOS_token:
                terminates.append(([int(idx) for idx in self.sentence_idxes] + [EOS_token],
                                   self.getScore())) # tuple(word_list, score_float)
                continue
            idxes = self.sentence_idxes[:] # pass by value
            scores = self.sentence_scores[:] # pass by value
            idxes.append(topi[0][i])
            scores.append(topv[0][i])
            sentences.append(Sentence(decoder_hidden, topi[0][i], idxes, scores))
        return terminates, sentences

    def toWordScore(self, voc):
        words = []
        for i in range(len(self.sentence_idxes)):
            if self.sentence_idxes[i] == EOS_token:
                words.append(EOS_token)
            else:
                words.append(int(self.sentence_idxes[i]))
        if self.sentence_idxes[-1] != EOS_token:
            words.append(EOS_token)
        return (words, self.getScore())


In [35]:
def generate(input, length, encoder, decoder, beam_size, lamda=0.0, threshold=0, verbose=False):
    # encoding and decoding
    hid_init = encoder.init_hidden(batch_size = 1)
    out_enc, hid_enc = encoder.forward(input.view(1,-1),length.view(1),hid_init)
    hid_dec = hid_enc#[:rnnDecoder.n_layers]

    if beam_size==0:
        path = topOneDecode(decoder, hid_dec, dataStat, max_length=15)  # return path in list
        return path
    else:
        path_beam = beamDecode(decoder, hid_dec, dataStat, beam_size, lamda, threshold)  # return list of tuples: (path, score)
        if verbose:
            for p in path_beam: print(float(p[1]), '\t', showResult(p[0]))
        return path_beam[0][0]


In [75]:
def evaluateSample(encoder, decoder, beam_size=5, lamda=0.0, threshold=0, verbose=True, myQuery=''):
    
    if myQuery == '':
        print("Blank Input")
        return -1
    else:
        # feed in customized tokens
        sample_query = myQuery.lower()
        sample_query_ind, _ = preprocess.encodePair(dataStat, (sample_query,'.'),reverse=True)
        sample_query_tensor = torch.LongTensor([padding(sample_query_ind, input_max_len)])
        sample_query_length = torch.LongTensor([len(sample_query_ind)])
        if USE_CUDA: input, length, target = sample_query_tensor.cuda(), sample_query_length.cuda(), None
        
    trace = generate(input, length, encoder, decoder, beam_size, lamda, threshold, verbose=True)
    if verbose:
        print("Message:\t", showResult(input.data[0].cpu().numpy(), reverse=True))
        print("Response:\t", showResult(trace))
        if target is not None:
            print("Teaching:\t", showResult(target.data[0].cpu().numpy()))
    return 0


In [37]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()

def evaluateCorpus(encoder, decoder, beam_size=1, lamda=0.0, threshold=0, loader=testLoader, display=False):
    
    loader.reset(1)
    data_length = loader.dataLength
    responses = []
    total_score = 0
    sample_length = min(2000, data_length)
    
    for i in range(sample_length):
        inputs, targets, lengths = loader.getMiniBatch()
        input, length, target = inputs, lengths[0][0], targets

        trace = generate(input, length, encoder, decoder, beam_size, lamda, threshold)
        responses.append(trace)
                
        length_ref = lengths[0][1]
        references = [[target.data[0].tolist()[:int(length_ref)]]]
        candidates = [trace]
        score = corpus_bleu(references, candidates, smoothing_function=chencherry.method1)
        total_score += score
        
        if display and (i+1)%int(sample_length/10)==0: print("complete",int(100*(i+1)/sample_length),"%")
        
    return total_score/i, responses

In [38]:
# distinct evaluation

import nltk

def distinctEval(all_paths):

    response_ugm = set([])
    response_bgm = set([])        
    response_len = sum([len(p) for p in all_paths])

    for path in all_paths:
        for u in path:
            response_ugm.add(u)
        for b in list(nltk.bigrams(path)):
            response_bgm.add(b)

    print("total length of response:", response_len)
    print("distinct unigrams:", len(response_ugm)/response_len)
    print("distinct bigrams:", len(response_bgm)/response_len)

In [100]:
evaluateSample(rnnEncoder, rnnDecoder, 5, 0.2, 2, True, 'seriously one of the best shows ! ! !')

-2.576613664627075 	 love it ! can t wait
-3.0073845386505127 	 blocked that guy .
-3.1148147583007812 	 blocked that guy . thanks
-3.1317696571350098 	 blocked that guy
-3.5388383865356445 	 can t wait for it
-3.5720245838165283 	 love it ! can t wait for the next one .
-3.677725076675415 	 true ! but i m there
-4.128273963928223 	 blocked that guy . thanks for the heads up
-4.14186954498291 	 blocked that guy . thanks dude
-4.328236103057861 	 blocked that guy . thanks for the heads
Message:	 seriously one of the best shows ! ! !
Response:	 love it ! can t wait


0

## Hyper-parameter tuning

In [31]:
# baseline -- gamma = 0, beam = 2
lamda_list = [0]
thres_list = [0]
score_list = [[],[],[]]

for threshold in thres_list:
    for lamda in lamda_list:
        print(time.asctime( time.localtime(time.time()) ))
        print(threshold, '\t', lamda)
        score_train,_ = evaluateCorpus(rnnEncoder, rnnDecoder,2, lamda, threshold, loader=trainLoader, display=0)
        score_deve,paths_deve = evaluateCorpus(rnnEncoder, rnnDecoder, 2, lamda, threshold, loader=deveLoader, display=0)
        score_test,paths_test = evaluateCorpus(rnnEncoder, rnnDecoder, 2, lamda, threshold, loader=testLoader, display=0)
        score_list[0].append(score_train)
        score_list[1].append(score_deve)
        score_list[2].append(score_test)
        print(score_train, score_deve, score_test)
        distinctEval(paths_deve)
        distinctEval(paths_test)

# score_list

Sat May 19 15:37:01 2018
0 	 0
0.6612963325219124 0.037271626386776724 0.03731144918329606
total length of response: 13744
distinct unigrams: 0.10702852153667054
distinct bigrams: 0.41916472642607683
total length of response: 13780
distinct unigrams: 0.10377358490566038
distinct bigrams: 0.4126269956458636


In [ ]:
# hyper-prrameter tuning -- gamma is set as 0.25
lamda_list = [-0.2,0,0.2,0.5]
thres_list = [1,2]
score_list = [[],[],[]]

for threshold in thres_list:
    for lamda in lamda_list:
        print(time.asctime( time.localtime(time.time()) ))
        print(threshold, '\t', lamda)
        score_train,_ = evaluateCorpus(rnnEncoder, rnnDecoder,2, lamda, threshold, loader=trainLoader, display=0)
        score_deve,paths_deve = evaluateCorpus(rnnEncoder, rnnDecoder, 2, lamda, threshold, loader=deveLoader, display=0)
        score_test,paths_test = evaluateCorpus(rnnEncoder, rnnDecoder, 2, lamda, threshold, loader=testLoader, display=0)
        score_list[0].append(score_train)
        score_list[1].append(score_deve)
        score_list[2].append(score_test)
        print(score_train, score_deve, score_test)
        distinctEval(paths_deve)
        distinctEval(paths_test)

# score_list

Sat May 19 12:59:29 2018
1 	 -0.2
0.7922088458092297 0.039162222479560166 0.03701055174145775
total length of response: 14590
distinct unigrams: 0.10095956134338588
distinct bigrams: 0.4038382453735435
total length of response: 14911
distinct unigrams: 0.10187110187110188
distinct bigrams: 0.40916102206424787
Sat May 19 13:05:17 2018
1 	 0
0.8278269493974896 0.03775382131805592 0.037414530302966875
total length of response: 14829
distinct unigrams: 0.10378312765527008
distinct bigrams: 0.4145930271764785
total length of response: 14859
distinct unigrams: 0.10532337303990848
distinct bigrams: 0.4200821051214752
Sat May 19 13:11:05 2018
1 	 0.2
0.8239186581560382 0.03774870156079711 0.037956076614411444
total length of response: 14767
distinct unigrams: 0.11173562673528814
distinct bigrams: 0.431908986253132
total length of response: 14662
distinct unigrams: 0.11219478925112536
distinct bigrams: 0.4288637293684354
Sat May 19 13:16:51 2018
1 	 0.5
0.8034878953760112 0.03598719670261189 0.

In [25]:
# hyper-prrameter tuning
lamda_list = [0.1, 0.3]  # gamma = 0.2
thres_list = [2, 3]
score_list = [[],[],[]]

for threshold in thres_list:
    for lamda in lamda_list:
        print(time.asctime( time.localtime(time.time()) ))
        print(threshold, '\t', lamda)
        score_train,_ = evaluateCorpus(rnnEncoder, rnnDecoder,5, lamda, threshold, loader=trainLoader, display=0)
        score_deve,paths_deve = evaluateCorpus(rnnEncoder, rnnDecoder, 5, lamda, threshold, loader=deveLoader, display=0)
        score_test,paths_test = evaluateCorpus(rnnEncoder, rnnDecoder, 5, lamda, threshold, loader=testLoader, display=0)
        score_list[0].append(score_train)
        score_list[1].append(score_deve)
        score_list[2].append(score_test)
        print(score_train, score_deve, score_test)
        distinctEval(paths_deve)
        distinctEval(paths_test)

# score_list

Sun May 20 17:16:09 2018
2 	 0.1
0.8709994793001742 0.04082476571928066 0.0450602922901014
total length of response: 12569
distinct unigrams: 0.1265017105577214
distinct bigrams: 0.43607287771501313
total length of response: 12468
distinct unigrams: 0.13089509143407121
distinct bigrams: 0.44048764837985244
Sun May 20 17:36:31 2018
2 	 0.3
0.8419612373889789 0.038244517589677354 0.040930512310572785
total length of response: 12240
distinct unigrams: 0.14910130718954248
distinct bigrams: 0.4812091503267974
total length of response: 12245
distinct unigrams: 0.14789710085749286
distinct bigrams: 0.4791343405471621
Sun May 20 17:56:03 2018
3 	 0.1
0.8611223711423327 0.03780930806804727 0.038418308167594076
total length of response: 12635
distinct unigrams: 0.12916501780767708
distinct bigrams: 0.4460625247328848
total length of response: 12549
distinct unigrams: 0.12957207745637103
distinct bigrams: 0.4450553828990358
Sun May 20 18:16:02 2018
3 	 0.3
0.8384625376860889 0.0383770596847083 0.

In [28]:
# hyper-prrameter tuning
lamda_list = [0.2, 0.4]  # gamma = 0.4
thres_list = [1, 2]
score_list = [[],[],[]]

for threshold in thres_list:
    for lamda in lamda_list:
        print(time.asctime( time.localtime(time.time()) ))
        print(threshold, '\t', lamda)
        score_train,_ = evaluateCorpus(rnnEncoder, rnnDecoder,5, lamda, threshold, loader=trainLoader, display=0)
        score_deve,paths_deve = evaluateCorpus(rnnEncoder, rnnDecoder, 5, lamda, threshold, loader=deveLoader, display=0)
        score_test,paths_test = evaluateCorpus(rnnEncoder, rnnDecoder, 5, lamda, threshold, loader=testLoader, display=0)
        score_list[0].append(score_train)
        score_list[1].append(score_deve)
        score_list[2].append(score_test)
        print(score_train, score_deve, score_test)
        distinctEval(paths_deve)
        distinctEval(paths_test)

# score_list

Sun May 20 13:51:23 2018
1 	 0.2
0.8599463678148406 0.03577600717412278 0.03621745163534304
total length of response: 15477
distinct unigrams: 0.11565548878981714
distinct bigrams: 0.43367577695935905
total length of response: 15451
distinct unigrams: 0.1179211701507993
distinct bigrams: 0.4388712704679309
Sun May 20 14:10:36 2018
1 	 0.4
0.8533362014076419 0.03697943346296261 0.03621004072653505
total length of response: 15289
distinct unigrams: 0.12433775917326183
distinct bigrams: 0.4378965269147753
total length of response: 15461
distinct unigrams: 0.13226828795032664
distinct bigrams: 0.457279606752474
Sun May 20 14:29:41 2018
2 	 0.2
0.8585377417994895 0.03939858055291495 0.042143563915589766
total length of response: 15352
distinct unigrams: 0.11933298593017197
distinct bigrams: 0.44482803543512245
total length of response: 15330
distinct unigrams: 0.12609262883235486
distinct bigrams: 0.45714285714285713
Sun May 20 14:49:57 2018
2 	 0.4
0.8217705027980924 0.03676383185325803 0.